In [46]:
import numpy as np

class site:
    def __init__(self, position, properties):
        self.index = position
        for name,value in properties.items():
            setattr(self, name, value)
        

class Ising:
    def __init__(self, inL, inh0):
        self.L = inL
        self.h0 = inh0
        self.rstep = 0
        self.system = []
        self.OmegaTrack = []
        self.hdist = [[0]]
        self.jdist = [[0]]
        self.lsdist = [[0]]
        self.lbdist = [[0]]
        self.mdist = [[0]]
        for i in range(1,self.L):
            self.system.append(site(i, {'Jright':np.random.uniform(0,1), 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
        self.system.append(site(self.L-1, {'Jright':0, 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
        
        self.hdist[0][0] = self.system[0].h
        self.jdist[0][0] = self.system[0].Jright
        self.lsdist[0][0] = self.system[0].ls
        self.lbdist[0][0] = self.system[0].lb
        self.mdist[0][0] = self.system[0].m
        for i in range(1,self.L-1):
            self.hdist[0].append(self.system[i].h)
            self.jdist[0].append(self.system[i].Jright)
            self.lsdist[0].append(self.system[i].ls)
            self.lbdist[0].append(self.system[i].lb)
            self.mdist[0].append(self.system[i].m)
        self.hdist[0].append(self.system[self.L-1].h)
        self.lsdist[0].append(self.system[self.L-1].ls)
        self.mdist[0].append(self.system[self.L-1].m)
    
    
    def renormStep(self):
        bond = None
        Omega = 0
        OmegaPos = None
        
        #finding max energy scale
        for i in range(self.L):
            if Omega<self.system[i].Jright:
                Omega = self.system[i].Jright
                OmegaPos = i
                bond = True
            if Omega<self.system[i].h:
                Omega = self.system[i].h
                OmegaPos = i
                bond = False
        self.OmegaTrack.append(Omega)
        
        #decimating
        if bond:
            self.system[OmegaPos].h = self.system[OmegaPos+1].h * self.system[OmegaPos].h / Omega #hright*hleft/Omega
            self.system[OmegaPos].ls = self.system[OmegaPos].ls + self.system[OmegaPos+1].ls + self.system[OmegaPos].lb
            self.system[OmegaPos].lb = self.system[OmegaPos+1].lb
            self.system[OmegaPos].m = self.system[OmegaPos].m + self.system[OmegaPos+1].m
            self.system[OmegaPos].Jright = self.system[OmegaPos+1].Jright
            del self.system[OmegaPos+1]
        elif OmegaPos<(self.L-1) and OmegaPos != 0:
            self.system[OmegaPos-1].Jright = self.system[OmegaPos-1].Jright*self.system[OmegaPos].Jright/Omega #jleft*jright/Omega
            self.system[OmegaPos-1].lb = self.system[OmegaPos-1].lb + self.system[OmegaPos].ls + self.system[OmegaPos].lb
            del self.system[OmegaPos]
        elif OmegaPos == 0:
            del self.system[OmegaPos]
        else:
            self.system[OmegaPos-1].Jright = 0
            del self.system[OmegaPos]
        
        #book keeping
        self.L -= 1
        self.rstep += 1
        
        #adding new distribution info
        self.hdist.append([0])
        self.hdist[self.rstep][0] = self.system[0].h
        self.jdist.append([0])
        self.jdist[self.rstep][0] = self.system[0].Jright
        self.lsdist.append([0])
        self.lsdist[self.rstep][0] = self.system[0].ls
        self.lbdist.append([0])
        self.lbdist[self.rstep][0] = self.system[0].lb
        self.mdist.append([0])
        self.mdist[self.rstep][0] = self.system[0].m
        for i in range(1,self.L-1):
            self.hdist[self.rstep].append(self.system[i].h)
            self.jdist[self.rstep].append(self.system[i].Jright)
            self.lsdist[self.rstep].append(self.system[i].ls)
            self.lbdist[self.rstep].append(self.system[i].lb)
            self.mdist[self.rstep].append(self.system[i].m)
        self.hdist[self.rstep].append(self.system[self.L-1].h)
        self.lsdist[self.rstep].append(self.system[self.L-1].ls)
        self.mdist[self.rstep].append(self.system[self.L-1].m)
        
    #Analytic Output
    def printCouplings(self):
        for i in range(self.L-1):
              print('h' + str(i) + ' = ' + str(self.system[i].h) + '\nJ' + str(i) + ',' + str(i+1) + ' = ' + str(self.system[i].Jright))
        print('h' + str(self.L-1) + ' = ' + str(self.system[self.L-1].h))
        print()
    
    def printDistributions(self):
        hdistlog = np.log(self.hdist[self.rstep])
        jdistlog = np.log(self.jdist[self.rstep])
        print('log average h is: ' + str(np.mean(hdistlog)))
        print('log h variance is: ' + str(np.var(hdistlog)) + '\n')
        print('log average J is: ' + str(np.mean(jdistlog)))
        print('log J variance is: ' + str(np.var(jdistlog)) + '\n')
        print('average spin cluster size is: ' + str(np.mean(self.lsdist[self.rstep])) + '\n')
        print('average bond length is: ' + str(np.mean(self.lbdist[self.rstep])) + '\n')
        print('average magnetic moment is: ' + str(np.mean(self.mdist[self.rstep]))+ '\n')
    
    def energyLengthScaling(self, startStep = 0, stopStep = None):
        if stopStep == None:
            stopStep = self.rstep
        VI = np.var(np.log(self.hdist[0])) + np.var(np.log(self.jdist[0]))
        for i in range(startStep, stopStep+1):
            print('log averaged h scales at renorm level ' + str(i) + ' with coefficient: ' + str(-np.mean(np.log(self.hdist[i]))/np.sqrt(VI * np.mean(self.lsdist[i]))) + '\n')
            print('log averaged J scales at renorm level ' + str(i) + ' with coefficient: ' + str(-np.mean(np.log(self.jdist[i]))/np.sqrt(VI * np.mean(self.lbdist[i]))) + '\n')
        
        
        
L0 = 2000
tester = Ising(L0,1)

#tester.printCouplings()
#tester.printDistributions()
for i in range(L0-10):
    tester.renormStep()
#tester.printCouplings()
#tester.printDistributions()
#tester.energyLengthScaling(1900,1950)

log averaged h scales at renorm level 1900 with coefficient: 1.8268015033790093

log averaged J scales at renorm level 1900 with coefficient: 1.7420700650996992

log averaged h scales at renorm level 1901 with coefficient: 1.822208391616765

log averaged J scales at renorm level 1901 with coefficient: 1.7487533167433527

log averaged h scales at renorm level 1902 with coefficient: 1.8267062964879137

log averaged J scales at renorm level 1902 with coefficient: 1.7464677805696478

log averaged h scales at renorm level 1903 with coefficient: 1.8275505849520841

log averaged J scales at renorm level 1903 with coefficient: 1.7477449354013135

log averaged h scales at renorm level 1904 with coefficient: 1.8237749280038464

log averaged J scales at renorm level 1904 with coefficient: 1.7534387395815838

log averaged h scales at renorm level 1905 with coefficient: 1.826296040507664

log averaged J scales at renorm level 1905 with coefficient: 1.7527626300959265

log averaged h scales at renor